In [ ]:
!pip install pypdf sentence-transformers faiss-cpu transformers torch
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 85.7 MB/s eta 0:00:00


In [ ]:
# ===============================
# IMPORT LIBRARIES
# ===============================
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import sys

# ===============================
# STEP 1: LOAD PDF (ERROR SAFE)
# ===============================
file_path = "2019BurkovTheHundred-pageMachineLearning.pdf"

def load_pdf(path):
    text = ""

    try:
        reader = PdfReader(path, strict=False)  # ⭐ handles corrupted PDFs
    except Exception as e:
        print("❌ Error opening PDF:", e)
        return text

    for i, page in enumerate(reader.pages):
        try:
            content = page.extract_text()
            if content:
                text += content + "\n"
        except Exception:
            print(f"⚠️ Skipping page {i} (corrupted)")

    return text

print("📚 Loading book...")
book_text = load_pdf(file_path)

if len(book_text) == 0:
    print("❌ No text extracted. Check PDF file.")
    sys.exit()

print("✅ Book loaded!")
print("Characters:", len(book_text))


# ===============================
# STEP 2: SPLIT TEXT INTO CHUNKS
# ===============================
def chunk_text(text, chunk_size=220):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

chunks = chunk_text(book_text)
print("✅ Chunks created:", len(chunks))


# ===============================
# STEP 3: CREATE EMBEDDINGS (RAG)
# ===============================
print("🔎 Creating embeddings...")
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(chunks, show_progress_bar=True)


# ===============================
# STEP 4: STORE IN FAISS
# ===============================
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))
print("✅ FAISS index ready!")


# ===============================
# STEP 5: LOAD GENERATOR MODEL
# ===============================
print("🤖 Loading generator model...")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
print("✅ Generator ready!")


def generate_answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# ===============================
# RAG FUNCTION
# ===============================
def ask_rag(question):
    q_embed = embed_model.encode([question])
    distances, indices = index.search(np.array(q_embed), k=3)

    context = " ".join([chunks[i] for i in indices[0]])[:1200]

    prompt = f"""
Answer using ONLY the context below.

Context:
{context}

Question: {question}

Explain clearly in simple words.
"""
    return generate_answer(prompt)


# ===============================
# NON-RAG FUNCTION
# ===============================
def ask_nonrag(question):

    prompt = f"""
You are a machine learning expert and tutor.

Answer the question clearly and in detail.
Explain in simple words.
Give a complete explanation.

Question: {question}

Answer:
"""
    return generate_answer(prompt)


# ===============================
# MAIN LOOP (MODE + Q&A)
# ===============================
while True:

    print("\n==============================")
    print("Choose Mode:")
    print("1 → RAG (uses book context)")
    print("2 → Non-RAG (model knowledge)")
    print("Type 'quit' to close program")
    print("==============================")

    mode = input("Enter 1 or 2: ").lower()

    if mode == "quit":
        print("👋 Closing program...")
        sys.exit()

    if mode not in ["1", "2"]:
        print("❌ Invalid choice. Try again.")
        continue

    if mode == "1":
        print("\n✅ RAG mode selected")
    else:
        print("\n✅ Non-RAG mode selected")

    print("\nAsk questions")
    print("Type 'exit' → change mode")
    print("Type 'quit' → close program\n")

    while True:
        question = input("Ask: ").lower()

        if question == "exit":
            print("\n🔄 Returning to mode selection...")
            break

        if question == "quit":
            print("\n👋 Closing program...")
            sys.exit()

        if mode == "1":
            answer = ask_rag(question)
        else:
            answer = ask_nonrag(question)

        print("\n💡 Answer:\n", answer)

📚 Loading book...
✅ Book loaded!
Characters: 279922
✅ Chunks created: 224
🔎 Creating embeddings...


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

✅ FAISS index ready!
🤖 Loading generator model...


Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


✅ Generator ready!

Choose Mode:
1 → RAG (uses book context)
2 → Non-RAG (model knowledge)
Type 'quit' to close program
